In [3]:
pwd

'/home/bergmann/peterb/CV/200_Rosen/010_pygimli_pn_cylinder'

In [1]:
%%file mesh.geo

lc = 0.08;

Point(1) = {0, 160, 0, lc};
Point(2) = {160, 0, 0, lc};
Point(3) = {0, -160, 0, lc};
Point(4) = {-160, 0, 0, lc};

Point(5) = {0, 210, 0, lc};
Point(6) = {210, 0, 0, lc};
Point(7) = {0, -210, 0, lc};
Point(8) = {-210, 0, 0, lc};

Line(1) = {1, 2};
Line(2) = {2, 3};
Line(3) = {3, 4};
Line(4) = {4, 1};

Line(5) = {5, 6};
Line(6) = {6, 7};
Line(7) = {7, 8};
Line(8) = {8, 5};

Line Loop(9) = {1,2,3,4};
Line Loop(10) = {5,6,7,8};
Plane Surface(11) = {9, 10};

Extrude {0, 0, 40} {
  Surface{11};
}

//Physical Point(99) = {2,3,4,5}; // electrode marker (99)

Mesh.CharacteristicLengthMin = 15;
//Mesh.CharacteristicLengthMin = 7;

Overwriting mesh.geo


In [3]:
from subprocess import getoutput as sgo
import pygimli.meshtools.polytools as plc

def print2file(string,file='mesh.geo'):
    sgo('echo "'+string+'" >> '+file)
    

radius_inner = 160
radius_outer = 210
segments = 60
c_inner = plc.createCircle([0, 0], radius=radius_inner, segments=segments)
c_outer = plc.createCircle([0, 0], radius=radius_outer, segments=segments)

!rm mesh.geo
print2file('lc = 0.08;')


In [12]:
from subprocess import getoutput as sgo
import pygimli.meshtools.polytools as plc

def print2file(string,file='mesh.geo'):
    sgo('echo "'+string+'" >> '+file)
    

radius_inner = 160
radius_outer = 210
pipe_length = 300
segments = 40
characteristicLengthMin = 25
c_inner = plc.createCircle([0, 0], radius=radius_inner, segments=segments)
c_outer = plc.createCircle([0, 0], radius=radius_outer, segments=segments)

!rm mesh.geo
print2file('lc = 0.08;')

pointIdx = 1
points_outer = []
for node in c_outer.nodes():
    print2file('Point('+str(pointIdx)+') ={'+str(node.x())+', '+\
                                        str(node.y())+', '+\
                                        str(node.z())+', lc};')
    points_outer.append(pointIdx)
    pointIdx += 1
print2file(' ')

points_inner = []
for node in c_inner.nodes():
    print2file('Point('+str(pointIdx)+') ={'+str(node.x())+', '+\
                                        str(node.y())+', '+\
                                        str(node.z())+', lc};')
    points_inner.append(pointIdx)
    pointIdx += 1    

#Create line entities
lineIdx = 1
for i,pointIdx in enumerate(points_outer):
    if i == 0:
        startIdx = pointIdx
    if i < len(points_outer)-1:
        print2file('Line('+str(lineIdx)+') ={'+str(pointIdx)+', '+str(pointIdx+1)+'};')
        lineIdx += 1
    elif i == len(points_outer)-1:
        print2file('Line('+str(lineIdx)+') ={'+str(pointIdx)+', '+str(startIdx)+'};')
        lineIdx += 1
print2file(' ')        
        
for i,pointIdx in enumerate(points_inner):
    if i == 0:
        startIdx = pointIdx
    if i < len(points_inner)-1:
        print2file('Line('+str(lineIdx)+') ={'+str(pointIdx)+', '+str(pointIdx+1)+'};')
        lineIdx += 1
    elif i == len(points_outer)-1:
        print2file('Line('+str(lineIdx)+') ={'+str(pointIdx)+', '+str(startIdx)+'};')
        lineIdx += 1
        
#Create line loop entities
lineLoopOuterIdx = lineIdx
print2file('Line Loop('+str(lineLoopOuterIdx)+') = {'+str(points_outer)[1:-1]+'};')

lineIdx += 1
lineLoopInnerIdx = lineIdx
print2file('Line Loop('+str(lineLoopInnerIdx)+') = {'+str(points_inner)[1:-1]+'};')

lineIdx += 1
print2file('Plane Surface('+str(lineIdx)+') = {'+str(lineLoopOuterIdx)+', '+str(lineLoopInnerIdx)+'};')

print2file(' ')    

print2file('Extrude {0, 0, '+str(pipe_length)+'} {')
print2file('  Surface{'+str(lineIdx)+'};')
print2file('}')

print2file('Mesh.CharacteristicLengthMin = '+str(characteristicLengthMin)+';')

In [13]:
# http://gmsh.info/doc/texinfo/gmsh.html#Mesh-options-list
from pygimli.meshtools import readGmsh
import subprocess

subprocess.call(["gmsh", "-3", "-o", "mesh.msh", "mesh.geo"])
mesh = readGmsh('mesh.msh', verbose=False)
mesh.save('mesh.bms')
mesh.exportVTK('mesh')
print(mesh)

Mesh: Nodes: 2684 Cells: 9052 Boundaries: 20432


In [14]:
!paraview --data=mesh.vtk

In [15]:
#Set boundary conditions
import numpy as np
import pygimli as pg

outer_boundaries = 0
for bound in mesh.boundaries():
    try:
        bound.leftCell().id()
        existLeftCell = True    
    except:
        existLeftCell = False

    try:
        bound.rightCell().id()
        existRightCell = True    
    except:
        existRightCell = False

    if np.array([existLeftCell,existRightCell]).all() == False:
        bound.setMarker(pg.MARKER_BOUND_HOMOGEN_NEUMANN)
        outer_boundaries += 1

mesh.save('mesh')
print(outer_boundaries)

4656


In [16]:
for i,node in enumerate(mesh.nodes()):
    if i == 240:
        elec1_x, elec1_y, elec1_z = node.x(),node.y(),node.z()
        print(elec1_x, elec1_y, elec1_z)
    if i == 641:
        elec2_x, elec2_y, elec2_z = node.x(),node.y(),node.z()
        print(elec2_x, elec2_y, elec2_z)        
    if i == 842:
        elec3_x, elec3_y, elec3_z = node.x(),node.y(),node.z()
        print(elec3_x, elec3_y, elec3_z)        
    if i == 843:
        elec4_x, elec4_y, elec4_z = node.x(),node.y(),node.z() 
        print(elec4_x, elec4_y, elec4_z)        

!echo "4# Number of electrodes" > config.shm
!echo "#x y z" >> config.shm
!echo "{elec1_x} {elec1_y} {elec1_z}" >> config.shm
!echo "{elec2_x} {elec2_y} {elec2_z}" >> config.shm
!echo "{elec3_x} {elec3_y} {elec3_z}" >> config.shm
!echo "{elec4_x} {elec4_y} {elec4_z}" >> config.shm
!echo "4# Number of data" >> config.shm
!echo "#a b m n" >> config.shm
!echo "1 2 3 4" >> config.shm
!echo "2 1 3 4" >> config.shm
!echo "1 4 3 2" >> config.shm
!echo "2 3 1 4" >> config.shm

!cat config.shm

210.0 0.0 24.99999999995
169.89356881875 -123.43490298132 149.999999999598
-94.04564036672 -129.4427191 224.999999999796
-94.04564036672 -129.4427191 249.999999999864
4# Number of electrodes
#x y z
210.0 0.0 24.99999999995
169.89356881875 -123.43490298132 149.999999999598
-94.04564036672 -129.4427191 224.999999999796
-94.04564036672 -129.4427191 249.999999999864
4# Number of data
#a b m n
1 2 3 4
2 1 3 4
1 4 3 2
2 3 1 4


In [17]:
#Set electrode marker
for node in mesh.nodes():
    if (node.x() == elec1_x) and \
        (node.y() == elec1_y) and \
        (node.z() == elec1_z):
            print(node.id())
            node.setMarker(99)  
    if (node.x() == elec2_x) and \
        (node.y() == elec2_y) and \
        (node.z() == elec2_z):
            print(node.id())        
            node.setMarker(99)
    if (node.x() == elec3_x) and \
        (node.y() == elec3_y) and \
        (node.z() == elec3_z):
            print(node.id())        
            node.setMarker(99)
    if (node.x() == elec4_x) and \
        (node.y() == elec4_y) and \
        (node.z() == elec4_z):
            print(node.id())        
            node.setMarker(99)
    
mesh.save('mesh')

240
641
842
843


1

In [18]:
!cat out2.ohm

4
# x y z 
181.14556962026	4.1e-11	40
4.1e-11	-181.14556962026	40
-181.14556962026	-4.1e-11	40
-4.1e-11	181.14556962026	40
4
# a b m n rho rhoa k
1	2	3	4	0.00000000000000e+00	3.00000000000000e+02	-2.30540740799328e+01
2	1	3	4	0.00000000000000e+00	3.00000000000000e+02	2.30540740788698e+01
1	4	3	2	0.00000000000000e+00	3.00000000000000e+02	-2.30959203178683e+01
2	3	1	4	0.00000000000000e+00	3.00000000000000e+02	2.30540740788705e+01
0


In [19]:
from pygimli.solver import solve

def mixedBC(boundary, userData):
    sourcePos = userData['sourcePos']
    k = userData['k']
    r1 = boundary.center() - sourcePos
    # Mirror on surface at depth=0
    r2 = boundary.center() - pg.RVector3(1.0, -1.0, 1.0) * sourcePos
    r1A = r1.abs()
    r2A = r2.abs()

    n = boundary.norm()
    # need rho here !!!!!!!!!!!!!!!!!!!!!!!!!!!1

    if r1A > 1e-12 and r2A > 1e-12:
        return k * ((r1.dot(n)) / r1A * pg.besselK1(r1A * k) +
                    (r2.dot(n)) / r2A * pg.besselK1(r2A * k)) / \
            (pg.besselK0(r1A * k) + pg.besselK0(r2A * k))
    else:
        return 0.
    
def pointSource(cell, f, userData):
    sourcePos = userData['sourcePos']

    if cell.shape().isInside(sourcePos):
        f.setVal(cell.N(cell.shape().rst(sourcePos)), cell.ids())


sourcePosA = [elec1_x, elec1_y, elec1_z]
sourcePosB = [elec2_x, elec2_y, elec2_z]

k = 1e-3
sigma = 1
u1 = solve(mesh, a=sigma, b=sigma * k*k, f=pointSource,
          duB=[[-1, mixedBC]],
          userData={'sourcePos': sourcePosA, 'k': k},
          verbose=True)

#BOundary conditions!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
u2 = solve(mesh, a=sigma, b=sigma * k*k, f=pointSource,
          duB=[[-1, mixedBC]],
          userData={'sourcePos': sourcePosB, 'k': k},
          verbose=True)

u = u1 - u2

Mesh:  Mesh: Nodes: 2684 Cells: 9052 Boundaries: 20432
{'k': 0.001, 'sourcePos': [210.0, 0.0, 24.99999999995]}
('Asssemblation time: ', 0.617)
('Solving time: ', 0.088)
Mesh:  Mesh: Nodes: 2684 Cells: 9052 Boundaries: 20432
{'k': 0.001, 'sourcePos': [169.89356881875, -123.43490298132, 149.999999999598]}
('Asssemblation time: ', 0.40700000000000003)
('Solving time: ', 0.023)


In [20]:
mesh.save('mesh')
!echo 'SCALARS valuesC double 1' >> mesh.vtk
!echo 'LOOKUP_TABLE default' >> mesh.vtk
!echo {str(list(u)).replace('[','').replace(']','').replace('\n','').replace(',','')} >> mesh.vtk

!paraview --data=mesh.vtk